# Tutorial 7: Grandes modelos de lenguaje (LLMs)

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabián Villena](https://villena.cl/).
- Profesor Auxiliar: Martín Paredes

En este notebook, exploraremos el uso de grandes modelos de lenguaje (LLMs) para tareas de procesamiento de lenguaje natural, con hincapié en aplicaciones clínicas y de salud. Cubriremos cómo conectarse a APIs de LLMs, enviar consultas, procesar respuestas y algunos ejemplos de prompting y buenas prácticas.

In [1]:
!pip install -q --progress-bar off -U "datasets<3.0"

In [2]:
import requests
from openai import OpenAI
from pydantic import BaseModel
import json
import datasets
from sklearn.metrics import classification_report

## Autenticación

Para interactuar con un LLM, primero debemos autenticar nuestra aplicación. Esto generalmente implica obtener una clave API de un proveedor de LLM. Asegúrate de tener tu clave API a mano.

In [3]:
API_KEY = ...

In [4]:
#Usando las claves secretas de colab se ve de esta forma
# from google.colab import userdata
# API_KEY = userdata.get('API_KEY')

## API de OpenAI

A través de consultas HTTP a la API de OpenAI, podemos enviar solicitudes y recibir respuestas de un modelo de lenguaje. Aquí hay un ejemplo básico de cómo hacerlo en Python.

In [5]:
url = "https://models.villena.cl/v1/responses"
headers = {
    "Content-Type": "application/json",
}
headers["Authorization"] = f"Bearer {API_KEY}"
data = {
    "model": "openai/gpt-5-nano",
    "input": "Define qué es el procesamiento de lenguaje natural clínico.",
}
response = requests.post(url, headers=headers, json=data)
response.json()

{'id': 'resp_bGl0ZWxsbTpjdXN0b21fbGxtX3Byb3ZpZGVyOm9wZW5haTttb2RlbF9pZDo5ZjU1MDFkMC1lM2Y0LTQzNzEtOWNlZS0xM2JhY2ZlNDc5MzI7cmVzcG9uc2VfaWQ6cmVzcF8wYzlmOGU2NzRiODJiY2MxMDA2OTBhMzJiMTBkZjQ4MTk2OTU4ZWFjNDEzYTllZmVkZQ==',
 'created_at': 1762276017,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-5-nano-2025-08-07',
 'object': 'response',
 'output': [{'id': 'rs_0c9f8e674b82bcc100690a32b1a3488196b8e7a3889b6729ad',
   'summary': [],
   'type': 'reasoning',
   'content': None,
   'encrypted_content': None,
   'status': None},
  {'id': 'msg_0c9f8e674b82bcc100690a32bbdad88196a2d0dcd62af944c7',
   'content': [{'annotations': [],
     'text': 'Definición breve:\nEl procesamiento de lenguaje natural clínico es la aplicación de técnicas de procesamiento de lenguaje natural (NLP) a textos clínicos no estructurados (historias clínicas, notas de evolución, informes de laboratorio, informes de radiología, etc.) para extraer, estructurar y analizar inform

## Uso de la biblioteca `openai`

En vez de hacer solicitudes HTTP manualmente, podemos usar la biblioteca `openai` para simplificar el proceso. Asegúrate de instalarla primero:


In [6]:
client = OpenAI(
    api_key=API_KEY,
    base_url="https://models.villena.cl",
)

###  Uso básico

Podemos usar la biblioteca `openai` para enviar solicitudes a la API de OpenAI. Aquí hay un ejemplo básico de cómo hacerlo

In [7]:
response = client.responses.create(
    model="openai/gpt-5-nano",
    instructions="Eres un experto en procesamiento de lenguaje natural clínico. Responde a la pregunta de manera clara y concisa.",
    input="¿Qué debo tomar en cuenta para desarrollar una función de preprocesamiento?",
)

print(response.output_text)

Aquí tienes una guía compacta y práctica para diseñar una función de preprocesamiento en NLP clínico (en español).

Qué definir primero
- Objetivo y salida: ¿qué tarea sigue (extracción de entidades, clasificación, similitud, etc.) y qué formato de salida necesitas (tokens, lemas, POS, entidades, indicadores de negación, etc.)?
- Tipo de datos y lenguaje: notas clínicas, informes de alta, historias clínicas; español (con posibles variaciones regionales, acentos y jerga médica).

Aspectos clave del preprocesamiento
- Normalización y limpieza
  - Mantener consistencia: decidir si convertir a minúsculas.
  - Normalizar espacios, saltos de línea y caracteres ilegibles.
  - Manejo de unidades y valores numéricos (unificar formatos, decimales, unidades).
  - Manejo de fechas y horas (normalizar formatos para facilitar aná­lisis temporal).
- Tokenización adecuada
  - Usar un tokenizador compatible con español y el dominio médico (SpaCy, Stanza, etc.).
  - Manejar términos médicos compuestos, 

También podemos usar imágenes como parte de nuestras consultas. A continuación, se muestra un ejemplo de cómo enviar una imagen junto con un mensaje de texto.


In [8]:
prompt = "¿Qué enfermedad es probable que tenga el paciente?"
img_url = "https://patoral.umayor.cl/canmucor/ca_leng_mb1.jpg"

response = client.responses.create(
    model="openai/gpt-5",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"{img_url}"},
            ],
        }
    ],
)

print(response.output_text)

Carcinoma epidermoide de la cavidad oral (probablemente de la lengua).


### Mensajes y roles

Los mensajes en la API de OpenAI tienen roles que indican quién está hablando. Los roles comunes son `system`, `user` y `assistant`. Aquí hay un ejemplo de cómo estructurar un mensaje

In [9]:
response = client.responses.create(
    model="openai/gpt-5-nano",
    input=[
        {"role": "developer", "content": "Eres un experto en salud digital."},
        {"role": "user", "content": "Qué es lo más importante al implementar un proyecto de informática médica?"}
    ]
)
print(response.output_text)


Lo más importante al implementar un proyecto de informática médica es asegurar que aporte valor clínico real sin perder de vista la seguridad, la interoperabilidad y la adopción por parte de los usuarios. En la práctica, esto se traduce en varios elementos clave que deben estar bien alineados desde el inicio:

- Definición clara del problema y valor clínico
  - Especificar el objetivo clínico medible (ej., reducción de errores, ahorro de tiempo, mejora de la seguridad del paciente).
  - Asegurarse de que el proyecto aborda una necesidad real y prioritaria de los equipos clínicos.

- Participación de usuarios y gobernanza
  - Involucrar a médicos, enfermeros, personal técnico y gestores desde el inicio (co-diseño, pruebas de usabilidad).
  - Establecer un comité de gobernanza de datos y un sponsor clínico.

- Seguridad, cumplimiento y ética
  - Diseñar con seguridad y privacidad por defecto (seguridad por diseño, minimización de datos, control de accesos, registro de auditoría).
  - Cum

In [10]:
developer_message = """
# Identidad

Eres un priorizador de la lista de espera chilena y deben priorizar pacientes en las categorías Urgente o Rutina.

# Instrucciones

* Solo responde con una palabra: "Urgente" o "Rutina".
"""

response = client.responses.create(
    model="openai/gpt-5-nano",
    input=[
        {"role": "developer", "content": developer_message},
        {"role": "user", "content": "El paciente presenta un dolor muy leve en el dedo meñique izquierdo."}
    ]
)
print(response.output_text)


Rutina


### Adición de ejemplos

Para mejorar la calidad de las respuestas, podemos proporcionar ejemplos de preguntas y respuestas esperadas. Esto ayuda al modelo a entender mejor el contexto y las expectativas.

In [11]:
developer_message = """
# Identidad

Eres un priorizador de la lista de espera chilena y deben priorizar pacientes en las categorías Urgente o Rutina.

# Instrucciones

* Solo responde con una palabra: "Urgente" o "Rutina".

# Ejemplos

<interconsulta>
El paciente presenta pérdida de peso de 10 kg en los últimos 3 meses y tiene antecedentes de cáncer de colon.
</interconsulta>
<assistant_response">
Urgente
</assistant_response>
<interconsulta>
El paciente presenta un dolor muy leve en el dedo meñique izquierdo.
</interconsulta>
<assistant_response">
Rutina
</assistant_response>
<interconsulta>
El paciente tiene un dolor intenso en el pecho y dificultad para respirar.
</interconsulta>
<assistant_response">
Urgente
</assistant_response>
"""

response = client.responses.create(
    model="openai/gpt-5-nano",
    input=[
        {"role": "developer", "content": developer_message},
        {"role": "user", "content": "El paciente tiene un dolor intenso en la zona parietal izquierda."}
    ]
)
print(response.output_text)


Urgente


### Respuesta estructurada

Para obtener respuestas más estructuradas, podemos usar el parámetro `text_format` en la solicitud. Esto nos permite especificar el formato de la respuesta esperada.

In [12]:
# Definir el esquema de salida estructurada
class InterconsultaPrioridad(BaseModel):
    prioridad: str  # "Urgente" o "Rutina"

# Usar el método parse para obtener una respuesta estructurada
response = client.responses.parse(
    model="openai/gpt-5-nano",
    input=[
        {"role": "system", "content": "Eres un priorizador de la lista de espera chilena. Responde solo con la prioridad del paciente ('Urgente' o 'Rutina')."},
        {"role": "user", "content": "El paciente tiene fiebre alta y dificultad respiratoria."}
    ],
    text_format=InterconsultaPrioridad,
)

print(response.output_parsed.prioridad)

Urgente


En algunos casos, es fundamental que la respuesta del modelo siga una estructura específica y validable, especialmente para integraciones clínicas o flujos automatizados. La API de OpenAI permite definir un esquema JSON (JSON Schema) que el modelo debe seguir estrictamente en su respuesta. A continuación, se muestra cómo definir un esquema para priorización de interconsultas y cómo solicitar al modelo que responda usando exactamente ese formato estructurado.

In [13]:
schema = {
    "type": "object",
    "properties": {
        "prioridad": {
            "type": "string",
            "enum": ["Urgente", "Rutina"],
            "description": "Prioridad del paciente según la interconsulta"
        },
        "descripcion": {
            "type": "string",
            "description": "Descripción adicional de la interconsulta"
        }
    },
    "required": ["prioridad", "descripcion"],
    "additionalProperties": False
}

response = client.responses.create(
    model="openai/gpt-5-nano",
    input=[
        {"role": "system", "content": "Eres un priorizador de la lista de espera chilena. Responde solo con la prioridad del paciente ('Urgente' o 'Rutina')."},
        {"role": "user", "content": "El paciente refiere dolor abdominal leve desde hace 2 semanas."}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "prioridad_interconsulta",
            "schema": schema,
            "strict": True
        }
    }
)

json.loads(response.output_text)

{'prioridad': 'Rutina',
 'descripcion': 'Dolor abdominal leve de 2 semanas sin signos de alarma.'}

### Chain of Thought (Cadena de Pensamiento)

El prompting tipo "chain of thought" (cadena de pensamiento) le indica al modelo que razone paso a paso antes de dar una respuesta final. Esto es útil para tareas complejas donde se requiere justificar o explicar el razonamiento detrás de la decisión.

A continuación, se muestra un ejemplo donde se le pide al modelo que explique su razonamiento antes de priorizar la interconsulta


In [14]:
cot_prompt = """
Eres un priorizador de la lista de espera chilena.
Primero, analiza los síntomas del paciente paso a paso y explica tu razonamiento.
Luego, responde con la prioridad final: "Urgente" o "Rutina".

Paciente: El paciente presenta fiebre alta, tos persistente y dificultad respiratoria.
"""

response = client.responses.create(
    model="openai/gpt-5-nano",
    input=[
        {"role": "system", "content": cot_prompt}
    ]
)

print(response.output_text)

Análisis paso a paso:

- Paso 1: Identificación de síntomas clave.
  - Fiebre alta: indica infección probable.
  - Tos persistente: señala infección respiratoria; puede ser viral o bacteriana.
  - Dificultad respiratoria: señal de compromiso respiratorio; riesgo de hipoxemia o deterioro rápido.

- Paso 2: Evaluación de gravedad y signos de alarma.
  - La disnea es un signo de alarma que sugiere necesidad de evaluación médica urgente.
  - No se proporcionan saturación de oxígeno ni signos vitales adicionales, pero la presencia de fiebre alta + disnea ya eleva la preocupación.

- Paso 3: Diagnóstico diferencial razonable.
  - Neumonía/bronquitis con compromiso respiratorio.
  - Infección viral severa (influenza, COVID-19 u otras).
  - Riesgo de complicaciones si hay comorbilidades no descritas (EPOC, asma, diabetes, cardiopatía).

- Paso 4: Implicaciones para la priorización.
  - Ante fiebre alta y dificultad respiratoria, es razonable clasificar como urgente para evitar retrasos en diag

## Clasificador utilizando LLMs

Los LLMs también pueden ser utilizados como clasificadores para tareas específicas, como la clasificación de interconsultas médicas. A continuación, se muestra un ejemplo de cómo implementar un clasificador utilizando un LLM.

In [16]:
import datasets
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics')

In [17]:
test = spanish_diagnostics['test'].select(range(10))

In [18]:
def classifier(text):
    schema = {
        "type": "object",
        "properties": {
            "tipo": {
                "type": "string",
                "enum": ["dental", "no_dental"],
                "description": "Tipo de diagnóstico: 'dental' cuando se debe enviar la interconsulta a una especialidad dental o 'no_dental' cuando no se debe enviar a una especialidad dental"
            }
        },
        "required": ["tipo"],
        "additionalProperties": False
    }
    response = client.responses.create(
        model="openai/gpt-5-nano",
        input=[
            {"role": "system", "content": "Eres un clasificador de diagnósticos médicos."},
            {"role": "user", "content": text}
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "diagnostico_clasificacion",
                "schema": schema,
                "strict": True
            }
        }
    )
    return json.loads(response.output_text)["tipo"]

In [19]:
classifier("caries en el diente 12 y 13, dolor leve al masticar")

'dental'

In [20]:
predicted = [classifier(item['text']) for item in test]

In [21]:
print(classification_report(["dental" if item['label'] == 1 else "no_dental" for item in test], predicted))

              precision    recall  f1-score   support

      dental       1.00      1.00      1.00         5
   no_dental       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [22]:
def classifier_few_shot(text):
    schema = {
        "type": "object",
        "properties": {
            "tipo": {
                "type": "string",
                "enum": ["dental", "no_dental"],
                "description": "Tipo de diagnóstico: 'dental' cuando se debe enviar la interconsulta a una especialidad dental o 'no_dental' cuando no se debe enviar a una especialidad dental"
            }
        },
        "required": ["tipo"],
        "additionalProperties": False
    }
    response = client.responses.create(
        model="openai/gpt-5-nano",
        input=[
            {"role": "system", "content": "Eres un clasificador de diagnósticos médicos."},
            {"role": "user", "content": "paciente con dolor en el diente 12 y 13"},
            {"role": "assistant", "content": '{"tipo":"dental"}'},
            {"role": "user", "content": "paciente con dolor en la rodilla derecha"},
            {"role": "assistant", "content": '{"tipo":"no_dental"}'},
            {"role": "user", "content": text}
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "diagnostico_clasificacion",
                "schema": schema,
                "strict": True
            }
        }
    )
    return json.loads(response.output[1].content[0].text)["tipo"]

In [23]:
classifier_few_shot("El paciente presenta dolor en el diente 12 y 13, con sensibilidad al frío y al calor.")

'dental'

In [24]:
predicted = [classifier_few_shot(item['text']) for item in test]

In [25]:
print(classification_report(["dental" if item['label'] == 1 else "no_dental" for item in test], predicted))

              precision    recall  f1-score   support

      dental       1.00      1.00      1.00         5
   no_dental       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

